In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
%load_ext tensorboard

In [2]:
data_path_and_labels = pd.read_csv("../mapped_data/xerces-1.3.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/xerces-1.3/xerces-1_3_0/src/org/w3c/dom/ProcessingInstruction.java.restrictedcontent.embed'

In [3]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xerces-1.3/xerces-1_3_0/src/org/w3c/dom/ProcessingInstruction.java.restrictedcontent.embed'

In [4]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xerces-1.3/xerces-1_3_0/src/org/w3c/dom/ProcessingInstruction.java.restrictedcontent'

In [5]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [6]:
d = pd.read_csv("../mapped_data/xerces-1.3.csv")
d['data_path'][0]

'dataset/promise/defect data/xerces-1.3/xerces-1_3_0/src/org/w3c/dom/ProcessingInstruction.java.restrictedcontent.embed'

In [7]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/xerces-1.3/xerces-1_3_0/src/org/w3c/dom/ProcessingInstruction.java.restrictedcontent.embed'

In [8]:
d

,data_path,bugs
0,../dataset/promise/defect data/xerces-1.3/xerc...,0
1,../dataset/promise/defect data/xerces-1.3/xerc...,0
2,../dataset/promise/defect data/xerces-1.3/xerc...,0
3,../dataset/promise/defect data/xerces-1.3/xerc...,2
4,../dataset/promise/defect data/xerces-1.3/xerc...,0
...,...,...
441,../dataset/promise/defect data/xerces-1.3/xerc...,2
442,../dataset/promise/defect data/xerces-1.3/xerc...,0
443,../dataset/promise/defect data/xerces-1.3/xerc...,2
444,../dataset/promise/defect data/xerces-1.3/xerc...,0


In [9]:
train, test = train_test_split(d, test_size=0.2)

In [10]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [11]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [12]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

5872.0

In [13]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [14]:
train_y

array([ 0,  0,  1,  3,  0,  0,  2,  0,  0,  1,  2,  0,  0,  2,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1,  3,  2,  0,  1,
        0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,
        0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0, 30,  0,  0,  2,  0,  2,
        0,  8,  0,  0,  0,  0,  0,  5,  0,  0,  1,  0,  0,  0,  0,  2,  0,
        0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,
        0,  0,  0,  0,  0

In [15]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [16]:
train_x.shape

(356, 7543)

In [17]:
# define the model
model = Sequential()
model.add(tf.keras.layers.Embedding(10671, 30, input_length=train_x.shape[1]))
model.add(tf.keras.layers.Flatten())
model.add(Dense(1, activation='relu'))
# compile the model
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [18]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
12/12 [==============================] - 1s 37ms/step - loss: 4.4277 - mse: 4.4277 - mae: 0.6442 - root_mean_squared_error: 2.1042 - mean_squared_logarithmic_error: 0.2414
Epoch 2/100
12/12 [==============================] - 0s 27ms/step - loss: 3.8982 - mse: 3.8982 - mae: 0.6117 - root_mean_squared_error: 1.9744 - mean_squared_logarithmic_error: 0.1870
Epoch 3/100
12/12 [==============================] - 0s 29ms/step - loss: 3.5197 - mse: 3.5197 - mae: 0.6289 - root_mean_squared_error: 1.8761 - mean_squared_logarithmic_error: 0.1810
Epoch 4/100
12/12 [==============================] - 0s 25ms/step - loss: 3.1769 - mse: 3.1769 - mae: 0.5897 - root_mean_squared_error: 1.7824 - mean_squared_logarithmic_error: 0.1681
Epoch 5/100
12/12 [==============================] - 0s 29ms/step - loss: 2.8458 - mse: 2.8458 - mae: 0.5139 - root_mean_squared_error: 1.6870 - mean_squared_logarithmic_error: 0.1445
Epoch 6/100
12/12 [==============================] - 0s 24ms/step - loss: 2.6045

12/12 [==============================] - 0s 28ms/step - loss: 0.2804 - mse: 0.2804 - mae: 0.3164 - root_mean_squared_error: 0.5295 - mean_squared_logarithmic_error: 0.0830
Epoch 45/100
12/12 [==============================] - 0s 32ms/step - loss: 0.2890 - mse: 0.2890 - mae: 0.2646 - root_mean_squared_error: 0.5376 - mean_squared_logarithmic_error: 0.0714
Epoch 46/100
12/12 [==============================] - 0s 23ms/step - loss: 0.2685 - mse: 0.2685 - mae: 0.1821 - root_mean_squared_error: 0.5182 - mean_squared_logarithmic_error: 0.0493
Epoch 47/100
12/12 [==============================] - 0s 27ms/step - loss: 0.2351 - mse: 0.2351 - mae: 0.2271 - root_mean_squared_error: 0.4848 - mean_squared_logarithmic_error: 0.0545
Epoch 48/100
12/12 [==============================] - 0s 24ms/step - loss: 0.2263 - mse: 0.2263 - mae: 0.2332 - root_mean_squared_error: 0.4757 - mean_squared_logarithmic_error: 0.0529
Epoch 49/100
12/12 [==============================] - 0s 31ms/step - loss: 0.2208 - mse:

12/12 [==============================] - 0s 23ms/step - loss: 0.1016 - mse: 0.1016 - mae: 0.1114 - root_mean_squared_error: 0.3188 - mean_squared_logarithmic_error: 0.0241
Epoch 89/100
12/12 [==============================] - 0s 23ms/step - loss: 0.1010 - mse: 0.1010 - mae: 0.1134 - root_mean_squared_error: 0.3178 - mean_squared_logarithmic_error: 0.0249
Epoch 90/100
12/12 [==============================] - 0s 25ms/step - loss: 0.1021 - mse: 0.1021 - mae: 0.0943 - root_mean_squared_error: 0.3195 - mean_squared_logarithmic_error: 0.0218
Epoch 91/100
12/12 [==============================] - 0s 25ms/step - loss: 0.1019 - mse: 0.1019 - mae: 0.0977 - root_mean_squared_error: 0.3193 - mean_squared_logarithmic_error: 0.0235
Epoch 92/100
12/12 [==============================] - 0s 24ms/step - loss: 0.1083 - mse: 0.1083 - mae: 0.1403 - root_mean_squared_error: 0.3291 - mean_squared_logarithmic_error: 0.0303
Epoch 93/100
12/12 [==============================] - 0s 24ms/step - loss: 0.1038 - mse:

In [19]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 21467), started 2:22:03 ago. (Use '!kill 21467' to kill it.)

In [20]:
score = model.evaluate(test_x, test_y)
score

3/3 [==============================] - 0s 7ms/step - loss: 0.6820 - mse: 0.6820 - mae: 0.4498 - root_mean_squared_error: 0.8259 - mean_squared_logarithmic_error: 0.1825


[0.6820372939109802,
 0.6820372939109802,
 0.44982969760894775,
 0.8258554935455322,
 0.1825246661901474]

In [21]:
data_path_and_labels = pd.read_csv("../mapped_data/xerces-1.3.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/xerces-1.3/xerces-1_3_0/src/org/w3c/dom/ProcessingInstruction.java.restrictedcontent.embed'

In [22]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xerces-1.3/xerces-1_3_0/src/org/w3c/dom/ProcessingInstruction.java.restrictedcontent.embed'

In [23]:
embeddings = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    embeddings.append(lines)

In [24]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,7533,7534,7535,7536,7537,7538,7539,7540,7541,7542
0,1187.0,3710.0,1308.0,3405.0,1308.0,5136.0,2343.0,1308.0,1017.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,508.0,1017.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,272.0,272.0,2150.0,5376.0,4218.0,1308.0,4218.0,2150.0,2150.0,5376.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,326.0,3427.0,337.0,3514.0,274.0,3432.0,494.0,2267.0,494.0,5584.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1022.0,4026.0,1017.0,3863.0,3905.0,3532.0,3905.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,327.0,327.0,4574.0,326.0,3931.0,736.0,3834.0,3931.0,1954.0,5624.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
442,1181.0,1181.0,1181.0,4243.0,1308.0,4243.0,522.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
443,328.0,1308.0,2376.0,1962.0,5794.0,1962.0,4273.0,1962.0,5844.0,1962.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
444,1237.0,1308.0,2376.0,1962.0,5794.0,1962.0,4273.0,1237.0,4314.0,1237.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,7533,7534,7535,7536,7537,7538,7539,7540,7541,7542
0,0,1187.0,3710.0,1308.0,3405.0,1308.0,5136.0,2343.0,1308.0,1017.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,508.0,1017.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,272.0,272.0,2150.0,5376.0,4218.0,1308.0,4218.0,2150.0,2150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,326.0,3427.0,337.0,3514.0,274.0,3432.0,494.0,2267.0,494.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1022.0,4026.0,1017.0,3863.0,3905.0,3532.0,3905.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,2,327.0,327.0,4574.0,326.0,3931.0,736.0,3834.0,3931.0,1954.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
442,0,1181.0,1181.0,1181.0,4243.0,1308.0,4243.0,522.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
443,2,328.0,1308.0,2376.0,1962.0,5794.0,1962.0,4273.0,1962.0,5844.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
444,0,1237.0,1308.0,2376.0,1962.0,5794.0,1962.0,4273.0,1237.0,4314.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
train, test = train_test_split(data, test_size=0.2)

In [27]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,7533,7534,7535,7536,7537,7538,7539,7540,7541,7542
400,0,279.0,5058.0,2479.0,494.0,5058.0,2426.0,326.0,5058.0,3284.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,0,1504.0,1504.0,4531.0,1484.0,5584.0,1308.0,4531.0,5584.0,5113.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,0,616.0,3381.0,1308.0,5120.0,2162.0,1308.0,3668.0,1308.0,5299.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,0,685.0,3420.0,1308.0,5144.0,2380.0,1308.0,3489.0,612.0,3304.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
301,0,1492.0,1492.0,4531.0,1484.0,5584.0,1308.0,4531.0,5584.0,5312.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,5,299.0,1962.0,2816.0,694.0,3082.0,1962.0,3083.0,299.0,3881.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280,0,1527.0,5335.0,4354.0,1308.0,3716.0,1308.0,5072.0,4354.0,1308.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0,1187.0,3710.0,1308.0,3405.0,1308.0,5136.0,2343.0,1308.0,1017.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0,1018.0,5376.0,542.0,5376.0,543.0,5376.0,544.0,3905.0,1270.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,7533,7534,7535,7536,7537,7538,7539,7540,7541,7542
22,0,336.0,1020.0,4930.0,1017.0,5837.0,3905.0,3240.0,1018.0,2583.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162,2,323.0,4143.0,5057.0,3905.0,3025.0,323.0,4533.0,313.0,4405.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319,0,1518.0,1518.0,4531.0,1484.0,5584.0,1308.0,4531.0,5584.0,5113.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119,1,1378.0,1962.0,187.0,3905.0,5670.0,3905.0,116.0,3905.0,290.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
215,0,297.0,3905.0,526.0,3905.0,532.0,3905.0,529.0,3905.0,533.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,0,580.0,3307.0,1308.0,4918.0,3326.0,5072.0,1763.0,1308.0,5080.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226,1,1318.0,3905.0,1328.0,1312.0,3132.0,566.0,2946.0,1318.0,1318.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,0,3793.0,3905.0,5495.0,1308.0,5498.0,3905.0,3905.0,3795.0,3278.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201,1,397.0,297.0,2734.0,303.0,2899.0,1312.0,3132.0,3905.0,418.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
Y_train = train[' bugs']
Y_test = test[' bugs']

In [30]:
train_y = Y_train.to_numpy()
test_y = Y_test.to_numpy()

In [31]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [32]:
X_train = train.values
X_test = test.values

In [33]:
train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [34]:
# Designing and initializing the model.
model = Sequential()
model.add(SimpleRNN(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(80, dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [35]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100)

Epoch 1/100
12/12 [==============================] - 2s 10ms/step - loss: 4.7152 - mse: 4.7152 - mae: 0.8250 - root_mean_squared_error: 2.1714 - mean_squared_logarithmic_error: 0.3664
Epoch 2/100
12/12 [==============================] - 0s 9ms/step - loss: 4.6761 - mse: 4.6761 - mae: 0.7582 - root_mean_squared_error: 2.1624 - mean_squared_logarithmic_error: 0.3406
Epoch 3/100
12/12 [==============================] - 0s 7ms/step - loss: 4.6072 - mse: 4.6072 - mae: 0.7814 - root_mean_squared_error: 2.1464 - mean_squared_logarithmic_error: 0.3391
Epoch 4/100
12/12 [==============================] - 0s 7ms/step - loss: 4.5737 - mse: 4.5737 - mae: 0.7597 - root_mean_squared_error: 2.1386 - mean_squared_logarithmic_error: 0.3297
Epoch 5/100
12/12 [==============================] - 0s 7ms/step - loss: 4.6425 - mse: 4.6425 - mae: 0.7429 - root_mean_squared_error: 2.1546 - mean_squared_logarithmic_error: 0.3364
Epoch 6/100
12/12 [==============================] - 0s 7ms/step - loss: 4.6438 - ms

12/12 [==============================] - 0s 7ms/step - loss: 4.4791 - mse: 4.4791 - mae: 0.5896 - root_mean_squared_error: 2.1164 - mean_squared_logarithmic_error: 0.2677
Epoch 46/100
12/12 [==============================] - 0s 7ms/step - loss: 4.3381 - mse: 4.3381 - mae: 0.5415 - root_mean_squared_error: 2.0828 - mean_squared_logarithmic_error: 0.2360
Epoch 47/100
12/12 [==============================] - 0s 7ms/step - loss: 4.4893 - mse: 4.4893 - mae: 0.5642 - root_mean_squared_error: 2.1188 - mean_squared_logarithmic_error: 0.2638
Epoch 48/100
12/12 [==============================] - 0s 6ms/step - loss: 4.4651 - mse: 4.4651 - mae: 0.5475 - root_mean_squared_error: 2.1131 - mean_squared_logarithmic_error: 0.2507
Epoch 49/100
12/12 [==============================] - 0s 7ms/step - loss: 4.2770 - mse: 4.2770 - mae: 0.5481 - root_mean_squared_error: 2.0681 - mean_squared_logarithmic_error: 0.2388
Epoch 50/100
12/12 [==============================] - 0s 7ms/step - loss: 4.5152 - mse: 4.515

12/12 [==============================] - 0s 7ms/step - loss: 4.4326 - mse: 4.4326 - mae: 0.4936 - root_mean_squared_error: 2.1054 - mean_squared_logarithmic_error: 0.2297
Epoch 90/100
12/12 [==============================] - 0s 8ms/step - loss: 4.3304 - mse: 4.3304 - mae: 0.4913 - root_mean_squared_error: 2.0810 - mean_squared_logarithmic_error: 0.2276
Epoch 91/100
12/12 [==============================] - 0s 8ms/step - loss: 4.4691 - mse: 4.4691 - mae: 0.5085 - root_mean_squared_error: 2.1140 - mean_squared_logarithmic_error: 0.2418
Epoch 92/100
12/12 [==============================] - 0s 7ms/step - loss: 4.3179 - mse: 4.3179 - mae: 0.5185 - root_mean_squared_error: 2.0780 - mean_squared_logarithmic_error: 0.2293
Epoch 93/100
12/12 [==============================] - 0s 9ms/step - loss: 4.1230 - mse: 4.1230 - mae: 0.4988 - root_mean_squared_error: 2.0305 - mean_squared_logarithmic_error: 0.2140
Epoch 94/100
12/12 [==============================] - 0s 8ms/step - loss: 4.3409 - mse: 4.340

In [36]:
score = model.evaluate(test_x, test_y)
score

3/3 [==============================] - 0s 3ms/step - loss: 0.7638 - mse: 0.7638 - mae: 0.3559 - root_mean_squared_error: 0.8740 - mean_squared_logarithmic_error: 0.1951


[0.7638254761695862,
 0.7638254761695862,
 0.3558727204799652,
 0.8739711046218872,
 0.1951315551996231]